## Mikroserwis

In [18]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

sys.path.append(str(Path("..").resolve()))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Przykładowe wywołanie mikroserwisu odpowiedzialnego za przewidywanie ceny

W celu zrealizowania predykcji przez mikroserwis użytkownik musi zdefiniować odpowiednią zawartość requesta (odpowiada ona strukurze zdefiniowanej w *service/model.py*). Warto zwrócić uwagę, że podawane ciało musi być już wstępnie przetworzone. Najwygodniej można tego dokonać wykorzystując zdefninowaną przez nas klasę *FeatueBuilder*, która była również wykorzystana w pipelinie ML do wstępnego przygotowania niezbędnych atrybutów. Zastosowano takie rozwiązanie, ponieważ każda próbka danych dotycząca danego ogłoszenia ma wiele odpowiadających rekordów w *sessions*, co drastycznie zwiększyło by ilość przesyłanych danych.

In [19]:
import requests

payload = {
    "property_type": "rental_unit",
    "room_type": "Entire home/apt",
    "accommodates": 3,
    "bathrooms": 1,
    "bedrooms": 1,
    "beds": 2,
    "host_response_time": "within an hour",
    "host_response_rate": 0.95,
    "host_acceptance_rate": 100,
    "host_is_superhost": 0,
    "host_identity_verified": 1,
    "review_scores_rating": 4.93,
    "number_of_reviews": 503,
    "minimum_nights": 2,
    "maximum_nights": 365,
    "instant_bookable": 1,
    "distance_to_centre": 2.53,
    "is_luxury": 0,
    "is_bathroom_shared": 0,
    "amenity_dishwasher": 1,
    "amenity_iron": 0,
    "amenity_toaster": 1,
    "amenity_oven": 1,
    "amenity_kitchen": 1,
    "amenity_microwave": 1,
    "amenity_crib": 0,
    "amenity_dining_table": 1,
    "amenity_free_dryer_in_unit": 1,
    "amenity_pack_n_playtravel_crib": 0,
    "amenity_count": 21,
    "description_sentiment": 0.35,
    "neighborhood_overview_sentiment": 0.71,
    "listing_views_ltm": 106,
    "conversion_rate_ltm": 0.091,
    "average_lead_time": 13.81,
    "average_booking_duration": 5.6,
    "price": 115.0
}

response = requests.post(url="http://127.0.0.1:8080/predict", json=payload)
print(f"Request status code: {response.status_code}")
print(f"Server response (price prediction): {response.text}")

Request status code: 200
Server response (price prediction): {"prediction":115.2}


Po wywołaniu endpointu */predict* użytkownik dostaje informację zwrotną na temat tego jak została wyceniona jego oferta na podstawie standardu kwatery. Porównanie cen, przewidzianej i rzeczywistej, daje wywołującemu informacje na temat obszaru, w którym znajduje się oferta. Jeśli różnice są bardzo duże to ogłoszenie prawdopodobnie nie jest źle wycenione, ale w regionie znajduje się bardzo dużo ofert, przez co wystawiający muszą walczyć o każdego klienta po przez zmniejszanie cen.

## Test A/B

Oznaczenia:
- $q_L$ - jakość modelu bazowego regresji liniowej.
- $q_F$ - jakośc modelu bardziej zaawansowanego lasu losowego.

### Hipotezy

- Hipoteza zerowa $H_0$ : $q_F$ $\leq$ $q_L$ - wyniki lasu losowego nie są lepsze niż regresji liniowej.
- Hipoteza alternatywna $H_1$ : $q_F$ $\gt$ $q_L$ - wyniki lasu losowego są lepsze niż regresji liniowej.

### Przyjęte parametry

- poziom istotności:  **$\alpha$ $=$ $0,05$**
- moc testu:           **$1$ $-$ $\beta$ $=$ $0,8$** <br>
Oraz na podstawie notatnika *models_03.ipynb*:
- rozmiar efektu **$MDE$ $=$ $5$** - (poprawa jakości modelu o 5 jednostki MAE jest wystarczająca, aby uznać hipotezę zerową) - **założenie biznesowe**
- odchylenie standardowe predykcji: **$std_F$ $=$ $25$** i **$std_L$ $=$ $50$** 

### Wyznaczanie liczby potrzebnych próbek

Do wyznaczenia liczby potrzebnych próbek posłuży test **t-Studenta**.

In [20]:
from statsmodels.stats.power import TTestIndPower
from scipy.stats import norm, ttest_ind
import numpy as np

alpha = 0.05
power = 0.8
mde = 5.0
std_l = 50.0
std_f = 25.0
std_pooled = np.sqrt((std_l**2 + std_f**2) / 2)

effect_size = mde / std_pooled
analysis = TTestIndPower()
n_samples = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power, ratio=1.0)

print(f"About {int(np.ceil(n_samples))} observations per model variant.")
print(f"Total sample size: {int(np.ceil(2 * n_samples))} observations.")

About 983 observations per model variant.
Total sample size: 1965 observations.


Tak duża liczba wynika z faktu, iż:
- skuteczność modeli nie różni się od siebie znacząco pod względem błędu MAE.
- dla modelu liniowego występują duże odchylenia standardowe predykcji, co wprowadza duży szum.

Potrzeba uśrednienia wielu rzeczywistych obserwacji modeli, aby móc wyciągnąć z analizy wymierne wnioski.  Dopiero taka wielkość próby gwarantuje moc testu na poziomie 0.8, pozwalając na wiarygodne odrzucenie hipotezy zerowej $H_0$ i potwierdzenie przewagi lasu losowego ($q_F$) nad modelem bazowym ($q_L$).

### Wyznaczanie wartości krytycznej

Do wyznaczenia wartości krytycznej posłużył wzór: $$\Delta_{crit} = z_{1-\alpha} \cdot SE$$
gdzie:
- **SE** - Błąd standardowy różnicy średnich odchyleń
- **$z_{1-\alpha}$** - Wartość krytyczna z rozkładu normalnego

In [21]:
se = np.sqrt((std_l**2 / n_samples) + (std_f**2 / n_samples))
z_crit = norm.ppf(1 - alpha)
delta_crit = z_crit * se

print(f"Standard error (SE): {se:.3f}")
print(f"Statistical multiplier (z): {z_crit:.3f}")
print(f"Critical value (Delta_crit): {delta_crit:.2f} MAE")

Standard error (SE): 1.784
Statistical multiplier (z): 1.645
Critical value (Delta_crit): 2.93 MAE


### Metodyka testu

W każdym zapytaniu do mikroserwisu zaszyty jest niewidoczny dla użytkownika wybór modelu, który przeprowadzi dla niego predykcję. Każdy model ma 50% szans na zostanie wybranym:
```python
random_number = random.uniform(0, 1)
if random_number < 0.5:
    model = BASE_MODEL     # Linear regression
else:
    model = ADVANCED_MODEL # Random forest
```

Przy tak dużej potrzebnej liczbie próbek zapewni to w przybliżeniu równomierne używanie modeli.

### Zbieranie logów

Mikroserwis zapisuje logi w strukturze:

*level,timestamp,model,prediction,real*

Są w nich zawarte kluczowe informacje: identyfikacja modelu, cena przewidziana przez model oraz cena rzeczywista.


### Dane do symulacji

Na potrzeby przykładowej ewaluacji zostały wygenerowane przykładowe logi z działania mikroserwisu. Do tego celu zostały użyte dane z dodatkowej paczki, która nie brała udziału w szkoleniu modelu.

### Ewaluacja testu

In [22]:
import pandas as pd

logs = pd.read_csv("../logs/service.log")

logs['abs_error'] = np.abs(logs['prediction'] - logs['real'])

group_L = logs[logs['model'] == 'base']['abs_error']
group_F = logs[logs['model'] == 'advanced']['abs_error']

print(f"Number of samples: Linear: {len(group_L)}, Random Forest: {len(group_F)}, Total: {len(group_L) + len(group_F)}")
print(f"MAE: Linear: {group_L.mean():.2f}, Random Forest: {group_F.mean():.2f}")
print(f"Std Dev: Linear: {group_L.std():.2f}, Random Forest: {group_F.std():.2f}")

Number of samples: Linear: 1123, Random Forest: 1144, Total: 2267
MAE: Linear: 40.98, Random Forest: 33.08
Std Dev: Linear: 50.13, Random Forest: 35.19


### Interpretacja wyników

#### Istotność statystyczna

Jeżeli (dla $\alpha = 0,05$):
- $\Delta_{observed} > \Delta_{crit}$ – różnica w jakości modeli jest na tyle duża, że wykracza poza obszar błędu statystycznego (szumu). Wynik uznajemy za istotny statystycznie, co pozwala na bezpieczne wyciąganie wniosków.
- $\Delta_{observed} \leq \Delta_{crit}$ – zaobserwowana poprawa mieści się w granicach naturalnych wahań predykcji. Wynik nie jest istotny statystycznie, co oznacza, że różnica może być dziełem przypadku.

In [23]:
data_sufficient = (len(group_L) >= n_samples) and (len(group_F) >= n_samples)
is_stat_significant = False

if data_sufficient:
    observed_delta = group_L.mean() - group_F.mean()

    se = np.sqrt((group_L.std()**2 / len(group_L)) + (group_F.std()**2 / len(group_F)))
    delta_crit = norm.ppf(1 - alpha) * se

    t_stat, p_value = ttest_ind(group_L, group_F, equal_var=False, alternative='greater')

    is_stat_significant = p_value < alpha

    print("--- Data Reliability ---")
    print(f"Sample size reached: {len(group_L)} per group (Required: {int(np.ceil(n_samples))})")
    print(f"Observed Improvement: {observed_delta:.2f} MAE")
    print(f"Statistical Threshold (Delta_crit): {delta_crit:.2f} MAE")
else:
    print(f"NOT sufficient data. Collected {len(group_L)}/{int(np.ceil(n_samples))} per variant.")


--- Data Reliability ---
Sample size reached: 1123 per group (Required: 983)
Observed Improvement: 7.90 MAE
Statistical Threshold (Delta_crit): 3.00 MAE


#### Przyjęcie hipotezy

Jeżeli (dla $\alpha$ = 0,05):
- **p_value $\lt$ $\alpha$** - hipoteza $H_0$ zostaje odrzucona na rzecz $H_1$, czyli test wykazał, że las losowy przynosi zakładaną poprawę.
- **p_value $\geq$ $\alpha$** - hipoteza $H_0$ zostaje przyjęta, czyli test wykazał, że las losowy nie istnieją wystarczające dowody, aby uznać las losowy za lepszy.


In [24]:
if data_sufficient:
    print("\n--- Hypothesis Verdict ---")
    if is_stat_significant:
        print(f"RESULT: Reject H0. Accept H1 - Random Forest is significantly better (p={p_value:.5f})")
        h1_accepted = True
    else:
        print(f"RESULT: Failed to reject H0. No significant proof that RF is better (p={p_value:.5f})")
        h1_accepted = False


--- Hypothesis Verdict ---
RESULT: Reject H0. Accept H1 - Random Forest is significantly better (p=0.00001)


#### Opłacalność biznesowa

Jeżeli (dla $MDE = 5,0$):
- $\Delta_{observed} \geq MDE$ – zaobserwowany zysk jakościowy osiągnął lub przekroczył założony próg opłacalności. Założenie biznesowe zostało spełnione, co w pełni uzasadnia wdrożenie modelu na produkcję.
- $\Delta_{observed} < MDE$ – model może być statystycznie lepszy, ale skala poprawy jest mniejsza niż wymagane 5 jednostek MAE. Założenie biznesowe nie zostało w pełni spełnione, co wymaga ponownej oceny zasadności wdrożenia.

In [25]:
if data_sufficient and h1_accepted:
    print("\n--- Business Impact ---")
    # mde = 5.0
    meets_mde = observed_delta >= mde

    if meets_mde:
        print(f"SUCCESS: The improvement {observed_delta:.2f} meets or exceeds MDE {mde}.")
        print("ACTION: Deploy the Random Forest model to 100% of production traffic.")
    else:
        print(f"PARTIAL SUCCESS: Model is better, but the improvement {observed_delta:.2f} is below MDE {mde}.")
        print("ACTION: Consult stakeholders before deployment.")


--- Business Impact ---
SUCCESS: The improvement 7.90 meets or exceeds MDE 5.0.
ACTION: Deploy the Random Forest model to 100% of production traffic.
